In [ ]:
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'

# import matplotlib.pyplot as plt

import torch
print(torch.__version__)
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn

from IPython.display import clear_output

import helper

# Batch Gradient Descent

In [ ]:
data_dir = '/Users/akshaykulkarni/flowers/train/'
batch_size = 1024
tf = transforms.Compose([transforms.Resize((255, 255)),
                         transforms.ToTensor(),
                        ])

train_dataset = datasets.ImageFolder(data_dir, transform = tf)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True, drop_last = True)

data_dir = '/Users/akshaykulkarni/flowers/test/'

test_dataset = datasets.ImageFolder(data_dir, transform = tf)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 32, shuffle = True, drop_last = True)

In [ ]:
images, labels = next(iter(train_loader))
print(images.shape)
images = images.view(-1, images.shape[0]).float()
print(images.shape)
images /= 255
labels = labels.float()

In [ ]:
# Dimensions of different variables
# W and dw - (n, 1)
# b and db - (1)
# images - (n, batch_size)
# labels - (batch_size)
# out and pred - (1, batch_size)
# loss - (1)
m = len(train_loader) * batch_size
n = 255 * 255 * 3
learning_rate = 2
W = torch.randn(n, 1).float()
b = torch.randn(1).float()
num_epochs = 10

In [ ]:
def binary_cross_entropy(A, Y):
    loss = (-1 / batch_size) * (torch.matmul(torch.log(A), Y) + torch.matmul(torch.log(1 - A), (1 - Y)))
    return loss

In [ ]:
def sigmoid(x):
    return (1 / (1 + torch.exp(-x)))

In [ ]:
def forward(x, y):
    out = sigmoid(torch.add(torch.matmul(W.t(), x), b))
    # size - (1, batch_size)
    loss = binary_cross_entropy(out, y)
    return loss, out

In [ ]:
def backward(A, Y, X):
    Y2 = Y.view(1, Y.shape[0])
    dz = A - Y2
    dw = torch.matmul(X, dz.t()) / batch_size
    db = torch.sum(dz) / batch_size
    db = db.reshape([1])
    assert(W.shape == dw.shape)
    assert(b.shape == db.shape)
    return dw, db

In [ ]:
# Batch Gradient Descent
loss_list = list()
for i in range(4000):
    # convert to GPU tensors
    if torch.cuda.is_available():
        images, labels = images.cuda(), labels.cuda()
        W, b = W.cuda(), b.cuda()

    loss, pred = forward(images, labels)
    epoch_loss = loss.item()
    loss_list.append(loss.item())
    dw, db = backward(pred, labels, images)
    # updating weights
    W = W - (learning_rate * dw)
    b = b - (learning_rate * db)
    
#     print(W)
#     print(b)
    if i % 20 == 0:
        print('Loss at epoch ', i, ' = ', epoch_loss)

In [ ]:
if torch.cuda.is_available():
    images = images.cuda()
    labels = labels.cuda()
    W = W.cuda()
    b = b.cuda()
    
loss, pred = forward(images, labels)
pred = (pred >= 0.5)

pred = pred.view(pred.shape[1])
mismatch = (torch.eq(pred, labels.byte()))
acc = torch.sum(mismatch.float())
acc = acc * 100 / 1024
print('Training accuracy = ', acc)

# acc = 0
# for batch_idx, (images, labels) in enumerate(test_loader):
#     if torch.cuda.is_available():
#         images = images.cuda()
#         labels = labels.cuda()
#         W = W.cuda()
#         b = b.cuda()
#     labels = labels.float()
#     images = images.view(-1, images.shape[0]).float()
#     images /= 255

#     loss, pred = forward(images, labels)
#     pred = (pred >= 0.5)

#     pred = pred.view(pred.shape[1])
#     mismatch = torch.sum(torch.eq(pred, labels.byte()))
#     acc += mismatch.item()

# acc /= (len(test_loader) * 32)
# print('Validation accuracy = ', acc * 100)

# Mini-Batch Gradient Descent

In [ ]:
data_dir = '/home/ivlabs/users/akshay/flowers/train/'
batch_size = 32
tf = transforms.Compose([transforms.Resize((255, 255)),
                         transforms.RandomHorizontalFlip(p = 0.5),
                         transforms.ToTensor(),
                        ])

train_dataset = datasets.ImageFolder(data_dir, transform = tf)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True, drop_last = True)

data_dir = '/home/ivlabs/users/akshay/flowers/test/'

test_dataset = datasets.ImageFolder(data_dir, transform = tf)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = True, drop_last = True)

print('No. of training images - ', len(train_loader) * batch_size)
print('No. of testing images - ', len(test_loader) * batch_size)

In [ ]:
# Dimensions of different variables
# W and dw - (n, 1)
# b and db - (1)
# images - (n, batch_size)
# labels - (batch_size)
# out and pred - (1, batch_size)
# loss - (1)
m = len(train_loader) * batch_size
n = 255 * 255 * 3
learning_rate = 1
# W = torch.randn(n, 1).float()
b = torch.randn(1).float()
# Xavier initialization of weights
W = torch.empty(n, 1)
nn.init.xavier_uniform_(W, gain=nn.init.calculate_gain('sigmoid'))
num_epochs = 500

In [ ]:
def binary_cross_entropy(A, Y):
    loss = (-1 / batch_size) * (torch.matmul(torch.log(A), Y) + torch.matmul(torch.log(1 - A), (1 - Y)))
    return loss

def sigmoid(x):
    return (1 / (1 + torch.exp(-x)))

def forward(x, y):
    out = torch.add(torch.matmul(W.t(), x), b)
    out = sigmoid(out)
    # size - (1, batch_size)
    loss = binary_cross_entropy(out, y)
    return loss, out

def backward(A, Y, X):
    Y2 = Y.view(1, Y.shape[0])
    dz = A - Y2
    dw = torch.matmul(X, dz.t()) / batch_size
    db = torch.sum(dz) / batch_size
    db = db.reshape([1])
    assert(W.shape == dw.shape)
    assert(b.shape == db.shape)
    return dw, db

In [ ]:
train_acc_list = list()
train_loss_list = list()
val_loss_list = list()
val_acc_list = list()
for i in range(num_epochs):
    train_loss = 0
    train_acc = 0
    # DW = torch.zeros(n, 1).float()
    # DB = torch.zeros(1).float()
    for batch_idx, (images, labels) in enumerate(train_loader):
        # convert to GPU tensors
        if torch.cuda.is_available():
            images, labels = images.cuda(), labels.cuda()
            W, b = W.cuda(), b.cuda()
            # DW, DB = DW.cuda(), DB.cuda()

        # convert to float and resize images to (n, batch_size) size
        labels = labels.float()
        images = images.view(-1, images.shape[0]).float()
        images /= 255

        loss, pred = forward(images, labels)
        train_loss += loss.item()

        dw, db = backward(pred, labels, images)
        # assert(DW.shape == dw.shape)
        # assert(DB.shape == db.shape)
        # DW += dw
        # DB += db

        pred = (pred >= 0.5)
        pred = pred.view(pred.shape[1])
        mismatch = torch.sum(torch.eq(pred, labels.byte()))
        train_acc += mismatch.item()

        W = W - (learning_rate * dw)
        b = b - (learning_rate * db)

    # DB /= len(train_loader)
    # DW /= len(train_loader)
    # # updating weights
    # W = W - (learning_rate * DW)
    # b = b - (learning_rate * DB)

    if i % 25 == 0:
        weights_name = 'weights_' + str(i) + '.pt'
        bias_name = 'bias_' + str(i) + '.pt'
        torch.save(W, weights_name)
        torch.save(b, bias_name)
#     print(W)
#     print(b)

    train_loss /= len(train_loader)
    train_loss_list.append(train_loss)
    # print(f'Training Loss at {i} is equal to {epoch_loss}')

    train_acc /= (len(train_loader) * batch_size)
    train_acc_list.append(train_acc * 100)
    # print('Training accuracy at epoch ', i, ' = ', acc * 100)

    val_loss = 0
    val_acc = 0
    for batch_idx, (images, labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
            W = W.cuda()
            b = b.cuda()
        labels = labels.float()
        images = images.view(-1, images.shape[0]).float()
        images /= 255

        loss, pred = forward(images, labels)
        val_loss += loss.item()
        pred = (pred >= 0.5)

        pred = pred.view(pred.shape[1])
        mismatch = torch.sum(torch.eq(pred, labels.byte()))
        val_acc += mismatch.item()

    val_loss /= len(test_loader)
    val_loss_list.append(val_loss)
    # print('Validation Loss at epoch ', i, ' = ', val_loss)

    val_acc /= (len(test_loader) * batch_size)
    val_acc_list.append(val_acc * 100)
    # print('Validation accuracy at epoch ', i, ' = ', val_acc * 100)
    print(f'Epoch : [{i} / {num_epochs}] | TL : {train_loss} | TA : {train_acc * 100} | VL : {val_loss} | VA : {val_acc * 100}')

In [ ]:
import csv

csvfile = "train_loss_list_500epochs.csv"
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for value in train_loss_list:
        writer.writerow([value])

csvfile = "val_loss_list_500epochs.csv"
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for value in val_loss_list:
        writer.writerow([value])

csvfile = "train_acc_list_500epochs.csv"
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for value in train_acc_list:
        writer.writerow([value])

csvfile = "val_acc_list_500epochs.csv"
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for value in val_acc_list:
        writer.writerow([value])

In [ ]:
import matplotlib.pyplot as plt
import csv
import numpy as np

with open('/home/ivlabs/Downloads/train_loss_list_500epochs.csv', 'r') as f:
    reader = csv.reader(f)
    train_loss_list2 = list(reader)

train_loss_list = list()
for i in train_loss_list2:
    for j in i:
        train_loss_list.append(float(j))
        
with open('/home/ivlabs/Downloads/train_acc_list_500epochs.csv', 'r') as f:
    reader = csv.reader(f)
    train_acc_list2 = list(reader)

train_acc_list = list()
for i in train_acc_list2:
    for j in i:
        train_acc_list.append(float(j))
        
with open('/home/ivlabs/Downloads/val_loss_list_500epochs.csv', 'r') as f:
    reader = csv.reader(f)
    val_loss_list2 = list(reader)

val_loss_list = list()
for i in val_loss_list2:
    for j in i:
        val_loss_list.append(float(j))
        
with open('/home/ivlabs/Downloads/val_acc_list_500epochs.csv', 'r') as f:
    reader = csv.reader(f)
    val_acc_list2 = list(reader)

val_acc_list = list()
for i in val_acc_list2:
    for j in i:
        val_acc_list.append(float(j))

iterations = np.arange(0, 101, 1)
plt.plot(iterations, train_loss_list, 'r')
plt.plot(iterations, train_acc_list, 'g')
plt.plot(iterations, val_loss_list, 'b')
plt.plot(iterations, val_acc_list, 'y')
plt.show()